In [10]:
# Import the data
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn import preprocessing 

# Models to use in our pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler



from sklearn.tree import DecisionTreeRegressor

# Preprocessing dependencies
from sklearn.model_selection import train_test_split


# Metrics
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
df = pd.read_csv('data/valentine_gift.csv')
df.head()


,Gender,Age,Relationship_Status,Budget,Occasion,Preference,Relationship_Length,Personal_Interest,Past_Gift_Item,Past_Gift_Reaction,Recent_Purchase_1,Recent_Purchase_2,Recent_Purchase_3,Best_Gift
0,Male,56,Engaged,Low,Just Because,Adventurous,<6 months,Music,Tech Gadget,Loved it,Meal Kit Sub,Gym Membership,Magazine Sub,Fashion Accessory
1,Female,58,Single,Low,Anniversary,Surprise,6-12 months,Technology,Chocolates,Loved it,Phone Charger,Smart Speaker,Processor Chip,Fashion Accessory
2,Female,21,Single,Low,Valentine's Day,Romantic,5+ years,Cooking,Fashion Accessory,Loved it,Magazine Sub,Streaming Sub,Meal Kit Sub,Flowers
3,Other,60,Married,Very High,Anniversary,Sentimental,6-12 months,Music,Chocolates,Loved it,Meal Kit Sub,Gym Membership,Magazine Sub,Personalized Gift
4,Male,56,Engaged,Low,Just Because,Adventurous,<6 months,Music,Tech Gadget,Loved it,Meal Kit Sub,Gym Membership,Magazine Sub,Fashion Accessory


In [21]:
df['Personal_Interest'].unique()

array(['Music', 'Technology', 'Cooking', 'Fashion', 'Art', 'Sports'],
      dtype=object)

In [13]:
# merge two samilar value to one vlaue
def mergeData(df, column_name, value_one, value_two):
    df[column_name] = df[column_name].replace(value_one, value_two)
    
# encode column's value
def hot_encode(df):
       categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
       encoder = OneHotEncoder(sparse_output=False)
       one_hot_encoded = encoder.fit_transform(df[categorical_columns])

       one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
       df_encoded = pd.concat([df, one_hot_df], axis=1)
       df_encoded = df_encoded.drop(categorical_columns, axis=1)

       return df_encoded
    
def label_encode(df):
# label_encoder object knows  
# how to understand word labels. 
    label_encoder = preprocessing.LabelEncoder() 
    
    # Encode labels in column 'species'. 
    df['Best_Gift_Encoded']= label_encoder.fit_transform(df['Best_Gift']) 
    
    df['Best_Gift_Encoded'].unique() 
    print(df[['Best_Gift', 'Best_Gift_Encoded']].head())

    return df['Best_Gift_Encoded']

def clean_data(df):
    
    # drop useless columns
    df = df.drop(columns = ['Past_Gift_Reaction'])

    # combine the samiliar values for few columns
    # mergeData(df, "Recipient_Gender", 'Non-Binary', 'Other')
    # mergeData(df, "Preference", "Sentimental", "Romantic")
    # mergeData(df, "Personal_Interest", "Sports", "Fitness")
    # mergeData(df, "Personal_Interest", "Gaming", "Technology")
    # mergeData(df, "Best_Gift", "Handmade Gift", "Personalized Gift")
    # mergeData(df, "Best_Gift", "Chocolates", "Chocolates & Flowers")
    # mergeData(df, "Best_Gift", "Flowers", "Chocolates & Flowers")
    # mergeData(df, "Best_Gift", "Jewelry", "Fashion Accessories")


    # split X and y
    # use hot encoding to encode some columns in x 
    X = df.drop(columns=['Best_Gift'])
    X = hot_encode (X)

    # use OrdinalEncoder to encode "Best_Gift" column and get y
    y = label_encode(df)

    
    return X, y

In [14]:
def r2_adj(X, y, model):
    """
    Calculates adjusted r-squared values

    Args:
    X: Independent variables, the data to fit
    y: dependent variable, the target data to try to predict
    model: The estimator or object to use to train the data

    Returns: adjusted r sqaured value accountign for number of predictors
    """
    r2 = model.score(X, y)
    n = X.shape[0]
    p = y.ndim


def model_generator(X, y):
    y = y.values.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

    models = {
        "RF": RandomForestClassifier(),
        'DT': DecisionTreeClassifier(),
        'SVC': SVC(),
        'AB': AdaBoostClassifier()
    }


    for  name, model in models.items():
        pipeline = Pipeline([
            ("Scale", StandardScaler(with_mean=False)),
            (name, model)
        ])

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        print("name:", name, "score:", pipeline.score(X_test, y_test))
        # print(accuracy_score(y_test, y_pred))

    


In [15]:
X, y = clean_data(df)

           Best_Gift  Best_Gift_Encoded
0  Fashion Accessory                  1
1  Fashion Accessory                  1
2            Flowers                  2
3  Personalized Gift                  3
4  Fashion Accessory                  1


In [16]:
model_generator(X, y)

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


name: RF score: 0.962
name: DT score: 0.972


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


name: SVC score: 0.956
name: AB score: 0.564


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [17]:
print(X.head(1))

   Age  Gender_Female  Gender_Male  Gender_Other  Relationship_Status_Dating  \
0   56            0.0          1.0           0.0                         0.0   

   Relationship_Status_Engaged  Relationship_Status_Married  \
0                          1.0                          0.0   

   Relationship_Status_Single  Budget_High  Budget_Low  ...  \
0                         0.0          0.0         1.0  ...   

   Recent_Purchase_3_Handbag  Recent_Purchase_3_Hat  \
0                        0.0                    0.0   

   Recent_Purchase_3_Headphones  Recent_Purchase_3_Magazine Sub  \
0                           0.0                             1.0   

   Recent_Purchase_3_Meal Kit Sub  Recent_Purchase_3_Necklace  \
0                             0.0                         0.0   

   Recent_Purchase_3_Phone Charger  Recent_Purchase_3_Processor Chip  \
0                              0.0                               0.0   

   Recent_Purchase_3_Smart Speaker  Recent_Purchase_3_Streaming

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Gender               2000 non-null   object
 1   Age                  2000 non-null   int64 
 2   Relationship_Status  2000 non-null   object
 3   Budget               2000 non-null   object
 4   Occasion             2000 non-null   object
 5   Preference           2000 non-null   object
 6   Relationship_Length  2000 non-null   object
 7   Personal_Interest    2000 non-null   object
 8   Past_Gift_Item       2000 non-null   object
 9   Past_Gift_Reaction   2000 non-null   object
 10  Recent_Purchase_1    2000 non-null   object
 11  Recent_Purchase_2    2000 non-null   object
 12  Recent_Purchase_3    2000 non-null   object
 13  Best_Gift            2000 non-null   object
dtypes: int64(1), object(13)
memory usage: 218.9+ KB


In [ ]:
def getBestGiftForTeamNine():
    ourData = {'Gender': ['Male', 'Male','Male', 'Male', 'Female', 'Female'],
             'Age': [35, 20,38, 23, 32, 38], 
             'Relationship_Status': ['Single', 'Dating','Married', 'Single', 'Married','Married'], 
             'Budget': ['Medium', 'Low','Low', 'Very High', 'Low', 'High'], 
             'Occasion': ['Birthday', 'Just Because', 'Just Because', 'Just Because', 'Birthday','Just Because'], 
             'Preference': ['Practical', 'Practical', 'Romantic','Surprise','Surprise','Surprise'], 
             'Relationship_Length': ['<6 months', '<6 months', '5+ years','<6 months', '5+ years', '5+ years'], 
             'Personal_Interest': ['Technology', 'Music', 'Cooking','Music', 'Sports', 'Fashion'],
             'Past_Gift_Item': ['Chocolates', 'Tech Gadget', 'Flowers','Personalized Gift', 'Tech Gadget','Personalized Gift'],
             'Past_Gift_Reaction': ['Liked it', 'Loved it', 'Disliked it','Liked it', 'Loved it', 'Loved it'],
             'Recent_Purchase_1': ['Headphones', 'Streaming Sub', 'Meal Kit Sub','Phone Charger', 'Hat', 'Handbag'],
             'Recent_Purchase_2': ['Hat', 'Phone Charger', 'Streaming Sub','Hat', 'Smart Speaker', 'Necklace'],
             'Recent_Purchase_3': ['Streaming Sub', 'Headphones', 'Phone Charger','Streaming Sub', 'Headphones', 'Headphones']}
    
    df = pd.DataFrame(ourData)
    X_our = hot_encode(df)

    print('X_our',X_our)
    
    model = DecisionTreeClassifier()

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_our)

    print(y_pred)
    
    

In [20]:
getBestGiftForTeamNine()

X_our    Age  Gender_Female  Gender_Male  Relationship_Status_Dating  \
0   35            0.0          1.0                         0.0   
1   20            0.0          1.0                         1.0   
2   38            0.0          1.0                         0.0   
3   23            0.0          1.0                         0.0   
4   32            1.0          0.0                         0.0   
5   38            1.0          0.0                         0.0   

   Relationship_Status_Married  Relationship_Status_Single  Budget_High  \
0                          0.0                         1.0          0.0   
1                          0.0                         0.0          0.0   
2                          1.0                         0.0          0.0   
3                          0.0                         1.0          0.0   
4                          1.0                         0.0          0.0   
5                          1.0                         0.0          1.0   

   Bu

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Past_Gift_Reaction_Disliked it
- Past_Gift_Reaction_Liked it
- Past_Gift_Reaction_Loved it
- Personal_Interest_Fashion Accessory
Feature names seen at fit time, yet now missing:
- Gender_Other
- Occasion_Anniversary
- Occasion_Valentine's Day
- Past_Gift_Item_Fashion Accessory
- Past_Gift_Item_Subscription Service
- ...
